# PROYECTO SOLUCIÓN DE PROBLEMA: Dots and Boxes
### Descripción del proyecto
En este proyecto se desarrollara la solucion del problema Dots and Boxes utilizando el algoritmo Minimax con poda alpha-beta. El objetivo es crear un programa que pueda jugar de manera inteligente contra un oponente humano o contra otro programa.

1. Presente el diseño del problema y del modelo.
### Descripción del problema:
Dots and Boxes es un juego para dos jugadores que se juega en una cuadrícula de puntos. Los jugadores van alternando turnos para dibujar líneas entre los puntos adyacentes. El objetivo es completar cuadros, y el jugador que complete un cuadro obtiene un punto y tiene derecho a jugar nuevamente. El juego termina cuando no quedan más líneas por dibujar, y el jugador con más puntos gana.
### Diseño del modelo:
- **Conjunto de estados:**
    - m=numeros de filas de puntos
    - n=numeros de columnas de puntos
    - Conjunto de puntos: $P = {(i, j) | 0 <= i < m, 0 <= j < n}$
    - Definicion del tablero como un grafo $T = (P, E)$ donde $E$ es el conjunto de aristas que representan las posibles líneas entre puntos adyacentes y P es el conjunto de vértices que representan los puntos en el tablero. O de mejor manera un tablero m por n seria $T_{m,n} = (P, E)$
    - Relacion de adyacencia: dos puntos son adyacentes si están en la misma fila o columna y no hay una línea entre ellos. Formalmente: 
    
        $p=(i, j)$ y $q=(k, l)$ son adyacentes si:
        - $i = k$ y $|j - l| = 1$ (misma fila, columnas adyacentes)
        - $j = l$ y $|i - k| = 1$ (misma columna, filas adyacentes)
    - Total de aristas en el tablero: $|E| = m(n-1) + (m-1)n$
    - $S(L, C, t)$ representa el estado del juego donde: 
        - $L$ es el conjunto de líneas dibujadas (aristas en el grafo)
        - $C$ asigna a cada caja completada el jugador que la completó (0, 1 o ninguno)
        - $t$ indica el turno del jugador actual (0 o 1)

- **Estado inicial:**

    Un tablero vacío sin líneas dibujadas, representado por un conjunto de puntos sin conexiones. $S_0 = (L_0, C_0, t_0)$ donde:
    - $L_0 = \emptyset$ (no hay líneas dibujadas)
    - $C_0 = \emptyset$ (no hay cajas completadas)
    - $t_0 = 0$ (el jugador 0 comienza)

- **Estados terminales:**

    Un estado terminal es cuando L=E, es decir, cuando todas las líneas posibles han sido dibujadas. En este punto, el juego termina y se determina el ganador contando las cajas completadas por cada jugador $S_t = (E, C_t, t_t)$ donde:
    - $E$ (todas las líneas han sido dibujadas)
    - $C_t$ asigna a cada caja completada el jugador que la completó
    - $t_t$ no es relevante ya que el juego ha terminado



- **Acciones:**
    - $(i, j) <-> (i, j+1)$ para líneas horizontales ya que conectan puntos adyacentes en la misma fila.
        - i puede variar de 1 a m
        - j puede variar de 1 a n-1

        Conjunto de acciones horizontales: $E_h = {(i, j, h) | 1 <= i < m, 1 <= j < n-1}$
    - $(i, j) <-> (i+1, j)$ para líneas verticales ya que conectan puntos adyacentes en la misma columna.
        - i puede variar de 1 a m-1
        - j puede variar de 1 a n

        Conjunto de acciones verticales: $E_v = {(i, j, v) | 1 <= i < m-1, 1 <= j < n}$
    - Conjunto total de acciones: $E = E_h \cup E_v$
    - Disponibles:
        - Inicial $A(S_0) = E$ (todas las acciones están disponibles)
        - En un estado intermedio, $A(S) = E - L$ (acciones que no han sido tomadas)
        - En un estado terminal, $A(S_t) = 0$ (no hay acciones disponibles)
- **Modelo de transición:**
    
    Al tomar una acción $a \in A(S)$, la función de transición se puede definir como:
    
    $T(S, a) = (L', C', t')$ donde:
    - $L' = L \cup {a}$
    - Se verifica si la acción completa alguna caja. Si es así, se asigna esa caja al jugador actual en $C'$ y se actualiza para reflejar cualquier caja completada por la acción $a$
    - $t' = t$ si se completó una caja, de lo contrario $t' = 1 - t$

- **Funcion de coste**

    Sea B el número total de cajas en el tablero, y $C_0$ y $C_1$ el número de cajas completadas por los jugadores 0 y 1 respectivamente.


    - $Score_0(S) = |b \in B | C(b) = 0|$
    - $Score_1(S) = |b \in B | C(b) = 1|$

    $U(s) = Score_0(S) - Score_1(S)$
    - El objetivo del jugador 0 es maximizar U(s) y el objetivo del jugador 1 es minimizar U(s).
    
    Propiedades:
    - Si $U(S) > 0$, el jugador 0 está en ventaja.
    - Si $U(S) < 0$, el jugador 1 está en ventaja.
    - Si $U(S) = 0$, el juego está empatado.

2. Implemente los componentes necesarios para solucionar el problema.

In [20]:
class Tablero:
    def __init__(self, filas, columnas):
        self.filas = filas

        self.columnas = columnas

        lista_vertices = set()  # utilizamos un conjunto para evitar duplicados, se crean los vertices con las coordenadas i,j donde i es la fila y j es la columna
        for i in range(self.filas + 1):
            for j in range(self.columnas + 1):
                lista_vertices.add((i, j))
        self.vertices = lista_vertices

        lista_aristas = set()
        for i in range(
            self.filas + 1
        ):  # Va has filas +1 para incluir el ultimo vertice
            for j in range(self.columnas):
                aristas = tuple(
                    sorted(((i, j), (i, j + 1)))
                )  # ordenamos los vertices para evitar duplicados se realiza i,j y i,j+1 para crear la arista horizontal
                lista_aristas.add(aristas)
        for i in range(self.filas):
            for j in range(
                self.columnas + 1
            ):  # Va has columnas +1 para incluir el ultimo vertice
                aristas = tuple(
                    sorted(((i, j), (i + 1, j)))
                )  # ordenamos los vertices para evitar duplicados se realiza i,j y i+1,j para crear la arista vertical
                lista_aristas.add(aristas)
        self.aristas = lista_aristas

        cajas = {}
        for i in range(self.filas):
            for j in range(self.columnas):
                top = tuple(sorted(((i, j), (i, j + 1))))
                bottom = tuple(sorted(((i + 1, j), (i + 1, j + 1))))
                left = tuple(sorted(((i, j), (i + 1, j))))
                right = tuple(sorted(((i, j + 1), (i + 1, j + 1))))
                cajas[(i, j)] = {top, bottom, left, right}
        self.cajas = cajas  # Realizado con Inteligencia Artificial para crear las cajas con sus respectivas aristas, se utiliza un diccionario donde la clave es la coordenada de la caja y el valor es un conjunto de las aristas que forman la caja


In [21]:
class Estado:
    def __init__(self, tablero, turno=1, aristas_dibujadas=None, propietario_caja=None):
        self.tablero = tablero
        self.turno = turno
        if aristas_dibujadas is None:
            self.aristas_dibujadas = set()
        else:
            self.aristas_dibujadas = aristas_dibujadas.copy()  # IA generativa nos ayudo a utilizar copy para evitar modificar el conjunto original, se crea un nuevo conjunto con las aristas dibujadas
        if propietario_caja is None:
            self.propietario_caja = {caja: 0 for caja in tablero.cajas}
        else:
            self.propietario_caja = propietario_caja.copy()  # IA generativa nos ayudo a utilizar copy para evitar modificar el diccionario original, se crea un nuevo diccionario con los propietarios de las cajas

    def acciones_disponibles(self):
        return self.tablero.aristas - self.aristas_dibujadas

    def realizar_accion(self, movimiento):
        nuevas_aristas = self.aristas_dibujadas.copy()
        nuevo_propietario = self.propietario_caja.copy()
        nuevas_aristas.add(movimiento)
        se_completo = False
        for caja, aristas_caja in self.tablero.cajas.items():
            if movimiento in aristas_caja:
                if aristas_caja.issubset(
                    nuevas_aristas
                ):  # IA generativa nos ayudo a utilizar issubset para verificar si todas las aristas de la caja están dibujadas, si es así se asigna el propietario de la caja al jugador actual
                    nuevo_propietario[caja] = self.turno
                    se_completo = True
        if se_completo:
            siguiente_turno = self.turno
        else:
            siguiente_turno = 3 - self.turno
        return Estado(
            self.tablero,
            turno=siguiente_turno,
            aristas_dibujadas=nuevas_aristas,
            propietario_caja=nuevo_propietario,
        )

    def es_terminal(self):
        return len(self.aristas_dibujadas) == len(self.tablero.aristas)

    def funcion_utilidad(self):
        utilidad = 0
        for propietario in self.propietario_caja.values():
            if propietario == 1:
                utilidad += 1
            elif propietario == 2:
                utilidad -= 1
        return utilidad


In [22]:
"""Codigo generado con ayuda de chatGPT para mostrar el tablero de una manera visual, se utiliza un ciclo para recorrer las filas y columnas del tablero, se imprime un punto para cada vértice y se verifica si hay una arista dibujada entre los vértices para imprimir una línea horizontal o vertical, también se verifica si hay un propietario de la caja para imprimir el número del jugador que la posee."""


def mostrar_tablero(estado):
    filas = estado.tablero.filas
    columnas = estado.tablero.columnas

    for i in range(filas + 1):
        linea = ""
        for j in range(columnas + 1):
            linea += "."  # vértice
            if j < columnas:
                if ((i, j), (i, j + 1)) in estado.aristas_dibujadas:
                    linea += "---"
                else:
                    linea += "   "
        print(linea)

        if i < filas:
            linea = ""
            for j in range(columnas + 1):
                if ((i, j), (i + 1, j)) in estado.aristas_dibujadas:
                    linea += "|"
                else:
                    linea += " "
                if j < columnas:
                    propietario = estado.propietario_caja[(i, j)]
                    if propietario != 0:
                        linea += f" {propietario} "
                    else:
                        linea += "   "
            print(linea)
    print("\n")

In [23]:
def minimax_alpha_beta(estado, profundidad, alpha, beta, maximizando):
    if estado.es_terminal():
        return estado.funcion_utilidad()
    elif profundidad == 0:
        return estado.funcion_utilidad()
    elif maximizando:
        valor = float("-inf")
        for a in estado.acciones_disponibles():
            nuevo_estado = estado.realizar_accion(a)
            valor_hijo = minimax_alpha_beta(
                nuevo_estado, profundidad - 1, alpha, beta, False
            )
            valor = max(valor, valor_hijo)
            alpha = max(valor, alpha)
            if alpha >= beta:
                break
    else:
        valor = float("inf")
        for a in estado.acciones_disponibles():
            nuevo_estado = estado.realizar_accion(a)
            valor_hijo = minimax_alpha_beta(
                nuevo_estado, profundidad - 1, alpha, beta, True
            )
            valor = min(valor, valor_hijo)
            beta = min(valor, beta)
            if alpha >= beta:
                break
    return valor

In [24]:
def mejor_accion(estado, profundidad, maximizando=True):
    if maximizando:
        mejor_valor = float("-inf")
    else:
        mejor_valor = float("inf")
    accion_optima = None
    for a in estado.acciones_disponibles():
        nuevo_estado = estado.realizar_accion(a)
        valor = minimax_alpha_beta(
            nuevo_estado, profundidad - 1, float("-inf"), float("inf"), not maximizando
        )
        if maximizando:
            if valor > mejor_valor:
                mejor_valor = valor
                accion_optima = a
        else:
            if valor < mejor_valor:
                mejor_valor = valor
                accion_optima = a
    if maximizando:
        jugador = "Max (Jugador 1)"
    else:
        jugador = "Min (Jugador 2)"
    return accion_optima, mejor_valor, jugador


In [25]:
# Ejemplo de uso:
filas = int(input("Ingresa la cantidad de filas: "))
columnas = int(input("Ingresa la cantidad de columnas: "))
tablero = Tablero(filas, columnas)
estado_inicial = Estado(tablero)
estado_actual = estado_inicial
profundidad = 3
maximizando = True

while not estado_actual.es_terminal(): 
    accion, valor, jugador = mejor_accion(estado_actual, profundidad, maximizando)
    estado_actual = estado_actual.realizar_accion(accion)  # Aplicar acción primero
    mostrar_tablero(estado_actual)  # Mostrar tablero actualizado
    print(f"{jugador} elige arista: {accion}, valor esperado: {valor}\n")
    if estado_actual.turno == 1:
        maximizando = True
    else:
        maximizando = False
        
print("Juego terminado.")
if estado_actual.funcion_utilidad() > 0:
    print("Jugador 1 gana!")
elif estado_actual.funcion_utilidad() < 0:
    print("Jugador 2 gana!")
else:
    print("Empate!")

.   .   .
         
.   .   .
    |    
.   .   .


Max (Jugador 1) elige arista: ((1, 1), (2, 1)), valor esperado: 0

.   .   .
         
.---.   .
    |    
.   .   .


Min (Jugador 2) elige arista: ((1, 0), (1, 1)), valor esperado: 0

.   .   .
         
.---.   .
    |    
.   .---.


Max (Jugador 1) elige arista: ((2, 1), (2, 2)), valor esperado: 0

.---.   .
         
.---.   .
    |    
.   .---.


Min (Jugador 2) elige arista: ((0, 0), (0, 1)), valor esperado: 0

.---.   .
        |
.---.   .
    |    
.   .---.


Max (Jugador 1) elige arista: ((0, 2), (1, 2)), valor esperado: 0

.---.---.
        |
.---.   .
    |    
.   .---.


Min (Jugador 2) elige arista: ((0, 1), (0, 2)), valor esperado: -1

.---.---.
        |
.---.   .
|   |    
.   .---.


Max (Jugador 1) elige arista: ((1, 0), (2, 0)), valor esperado: -1

.---.---.
        |
.---.   .
| 2 |    
.---.---.


Min (Jugador 2) elige arista: ((2, 0), (2, 1)), valor esperado: -1

.---.---.
|       |
.---.   .
| 2 |    
.---.